<a href="https://colab.research.google.com/github/rhumtea/trainModelML/blob/main/phong_alexnet_like_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Introduction:
- This project designs and train the Deep Learnign model.
- I choose the Natural Inmages with 8 classes from Kaggle.
- Link: https://www.kaggle.com/datasets/prasunroy/natural-images

In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import kagglehub

# Download the dataset from Kaggle and save in Google Colab Files
!kaggle datasets download -d prasunroy/natural-images

Dataset URL: https://www.kaggle.com/datasets/prasunroy/natural-images
License(s): CC-BY-NC-SA-4.0
natural-images.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# Unzip the natural-images.zip
with zipfile.ZipFile('natural-images.zip', 'r') as zip_ref:
  zip_ref.extractall('')

# Define path for original dataset
data_dir = 'natural_images'

In [ ]:
import shutil
import random

# Define path for split data
base_dir = 'data_split'

# Define path for train, validation and test directory
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Create directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [ ]:
# Split ratio
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

In [ ]:
# Split data
categories = os.listdir(data_dir)

for category in categories:
  category_path = os.path.join(data_dir, category)

  if not os.path.isdir(category_path): continue

  # Create subdirectories for each class in train and val files
  os.makedirs(os.path.join(train_dir, category), exist_ok=True)
  os.makedirs(os.path.join(val_dir, category), exist_ok=True)
  os.makedirs(os.path.join(test_dir, category), exist_ok=True)

  # Get all images in the category to split
  images = os.listdir(category_path)
  random.shuffle(images)

  # Find indices of image to split
  total_images = len(images)
  train_images_index = int(train_ratio * total_images)
  val_images_index = int((train_ratio + val_ratio) * total_images)

  # Split data by indices
  train_images = images[:train_images_index]
  val_images = images[train_images_index:val_images_index]
  test_images = images[val_images_index:]

  # Move images to train and val directories with folders as original file:
  for image in train_images:
    shutil.copy(os.path.join(category_path, image), os.path.join(train_dir, category))

  for image in val_images:
    shutil.copy(os.path.join(category_path, image), os.path.join(val_dir, category))

  for image in test_images:
    shutil.copy(os.path.join(category_path, image), test_dir)

In [ ]:
# Define train generator which includes augmentation to expand the dataset
# and make the model more roburst
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range = 30,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range=0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

In [ ]:
# Define validation and test generators with normalization
val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
# Set image height, width for resizing image from th original size (uniformity)
# Set batch_size for deciding how many images per batch during train and validation
img_height, img_width = 227, 227
batch_size = 32

In [ ]:
# Training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical' # because of one-hot encode labels
)

# Validation generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical'
)

# Test_save generator
test_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 6239 images belonging to 8 classes.
Found 1897 images belonging to 8 classes.
Found 1897 images belonging to 8 classes.


2. Check before create Model

In [ ]:
# Get the number of classes
num_classes = len(train_generator.class_indices)
print(num_classes)
class_names = [item for item in train_generator.class_indices]
print(class_names)

8
['airplane', 'car', 'cat', 'dog', 'flower', 'fruit', 'motorbike', 'person']


3. AlexNet50 - like - first try

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Normalization
from tensorflow.keras.optimizers import Adam

# Build CNN model
input_shape = (img_width, img_height, 3)
model = Sequential([
            Conv2D(64, kernel_size=11, strides= 4,
                            padding= 'valid', activation= 'relu',
                            input_shape= input_shape,
                            kernel_initializer= 'he_normal'),
            MaxPooling2D(pool_size=(3,3), strides= (2,2),
                                    padding= 'valid', data_format= None),
            Normalization()
            Conv2D(128, kernel_size=(5,5), strides= 1,
                            padding= 'same', activation= 'relu',
                            kernel_initializer= 'he_normal'),
            MaxPooling2D(pool_size=(3,3), strides= (2,2),
                                    padding= 'valid', data_format= None),
            Conv2D(256, kernel_size=(3,3), strides= 1,
                            padding= 'same', activation= 'relu',
                            kernel_initializer= 'he_normal'),
            Conv2D(256, kernel_size=(3,3), strides= 1,
                            padding= 'same', activation= 'relu',
                            kernel_initializer= 'he_normal'),
            Conv2D(128, kernel_size=(3,3), strides= 1,
                            padding= 'same', activation= 'relu',
                            kernel_initializer= 'he_normal'),
            MaxPooling2D(pool_size=(3,3), strides= (2,2),
                                    padding= 'valid', data_format= None),
            Flatten(),
            Dense(1024, activation= 'relu'),
            Dense(512, activation= 'relu'),
            Dense(128, activation= 'relu'),
            Dense(num_classes, activation= 'softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4. Model for natural images

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 55, 55, 64)          │          23,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 27, 27, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 27, 27, 128)         │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 13, 13, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 13, 13, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 13, 13, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 13, 13, 128)         │         295,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       4,719,616 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 8)                   │           1,032 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,719,624 (25.63 MB)

 Trainable params: 6,719,624 (25.63 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compile the model
model.compile(optimizer= tf.keras.optimizers.Adam(0.001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    validation_data = val_generator,
    epochs=10,
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


195/195 ━━━━━━━━━━━━━━━━━━━━ 476s 2s/step - accuracy: 0.3007 - loss: 1.8550 - val_accuracy: 0.6231 - val_loss: 1.0080
Epoch 2/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 497s 2s/step - accuracy: 0.6242 - loss: 1.0291 - val_accuracy: 0.7612 - val_loss: 0.6606
Epoch 3/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 465s 2s/step - accuracy: 0.7151 - loss: 0.7693 - val_accuracy: 0.8113 - val_loss: 0.5447
Epoch 4/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 502s 2s/step - accuracy: 0.7612 - loss: 0.6512 - val_accuracy: 0.8376 - val_loss: 0.4263
Epoch 5/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 493s 2s/step - accuracy: 0.7828 - loss: 0.5855 - val_accuracy: 0.8640 - val_loss: 0.3640
Epoch 6/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 499s 2s/step - accuracy: 0.7874 - loss: 0.5794 - val_accuracy: 0.8397 - val_loss: 0.4079
Epoch 7/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 505s 2s/step - accuracy: 0.7986 - loss: 0.5696 - val_accuracy: 0.8898 - val_loss: 0.3178
Epoch 8/10
 74/195 ━━━━━━━━━━━━━━━━━━━━ 4:23 2s/step - accuracy: 0.8309 - loss: 0.4653

-- Save the Model --


In [ ]:
# Save and Reuse Model
model.save('resnet50_model.keras')

-- Load the Model --

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('resnet50_model.keras')

5. Evaluate Model

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")